In [1]:
import keras
#Shakespearean text generator

#Import
shakespeare_url = "https://homl.info/shakespeare" # shortcut URL
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [6]:
import numpy as np
import tensorflow as tf
#Fit a Tokenizer
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True) #Can make false for words and automatic puncuation filtering
tokenizer.fit_on_texts(shakespeare_text)

[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

max_id = len(tokenizer.word_index)
print(max_id)
dataset_size = tokenizer.document_count
print(dataset_size)

#Attempting to split data well
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

#Split sequences into windows
n_steps = 100 #Tunable
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

#Flatten
dataset = dataset.flat_map(lambda window: window.batch(window_length))

#Set up batches
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

#Categorical input
dataset = dataset.map(
lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

#Prefetch data
dataset = dataset.prefetch(1)

39
1115394


In [8]:
# The model
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
        dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
        dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
        activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=20, steps_per_epoch=100)

Epoch 1/20
100/100 [==============================] - 27s 193ms/step - loss: 3.2141
Epoch 2/20
100/100 [==============================] - 20s 199ms/step - loss: 2.5042
Epoch 3/20
100/100 [==============================] - 20s 200ms/step - loss: 2.3418
Epoch 4/20
100/100 [==============================] - 20s 201ms/step - loss: 2.2099
Epoch 5/20
100/100 [==============================] - 20s 196ms/step - loss: 2.0856
Epoch 6/20
100/100 [==============================] - 20s 199ms/step - loss: 2.0100
Epoch 7/20
100/100 [==============================] - 20s 205ms/step - loss: 1.9309
Epoch 8/20
100/100 [==============================] - 20s 196ms/step - loss: 1.8903
Epoch 9/20
100/100 [==============================] - 20s 198ms/step - loss: 1.8572
Epoch 10/20
100/100 [==============================] - 20s 196ms/step - loss: 1.8355
Epoch 11/20
100/100 [==============================] - 20s 204ms/step - loss: 1.8018
Epoch 12/20
100/100 [==============================] - 20s 199ms/step - lo

In [9]:
#Preprocessing before passing to our model
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

X_new = preprocess(["How are yo"])
Y_pred = model.predict_classes(X_new)
print(tokenizer.sequences_to_texts(Y_pred + 1)[0][-1])

j:\School Files\[In progress] ECE612 Applied Machine Learning\venv_tf\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
u


In [11]:
#Generating the new text
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

print(complete_text("t", temperature=0.2))
print(complete_text("w", temperature=1))
print(complete_text("w", temperature=2))

there with when you are shall be consul.

coriolanu
words, that whod of but the prech?

melenius:
or it
w; nod'.
sid, corno yeet somt
sbrktur thim
con'. vh


In [16]:
#Stateful RNN

#Get data
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

#Model
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
        dropout=0.2, recurrent_dropout=0.2,
        batch_input_shape=[1, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
        dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
        activation="softmax"))
])

#A callback
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

#Go for it
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
model.fit(dataset, epochs=20, steps_per_epoch=100, callbacks=[ResetStatesCallback()])

Epoch 1/20
100/100 [==============================] - 15s 95ms/step - loss: 3.2561
Epoch 2/20
100/100 [==============================] - 9s 94ms/step - loss: 2.8207
Epoch 3/20
100/100 [==============================] - 11s 107ms/step - loss: 2.5692
Epoch 4/20
100/100 [==============================] - 11s 115ms/step - loss: 2.5001
Epoch 5/20
100/100 [==============================] - 11s 106ms/step - loss: 2.4749
Epoch 6/20
100/100 [==============================] - 11s 108ms/step - loss: 2.4269
Epoch 7/20
100/100 [==============================] - 11s 110ms/step - loss: 2.3472
Epoch 8/20
100/100 [==============================] - 11s 114ms/step - loss: 2.3113
Epoch 9/20
100/100 [==============================] - 11s 112ms/step - loss: 2.2387
Epoch 10/20
100/100 [==============================] - 11s 112ms/step - loss: 2.2037
Epoch 11/20
100/100 [==============================] - 10s 102ms/step - loss: 2.2072
Epoch 12/20
100/100 [==============================] - 10s 101ms/step - loss:

In [18]:
#Sentiment Analysis with a word-level model using IMDB data
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token

1646592/1641221 [==============================] - 0s 0us/step


In [21]:
#Load stuff as bytes for mobile
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples

Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]
Dl Completed...: 0 url [00:00, ? url/s]
Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]
Generating train examples...: 0 examples [00:00, ? examples/s]Downloading and preparing dataset Unknown size (download: Unknown size, generated: Unknown size, total: Unknown size) to C:\Users\Cameron James\tensorflow_datasets\imdb_reviews\plain_text\1.0.0...



Generating train examples...: 1 examples [00:02,  2.93s/ examples]
Generating train examples...: 106 examples [00:03, 49.11 examples/s]
Generating train examples...: 208 examples [00:03, 109.49 examples/s]
Generating train examples...: 304 examples [00:03, 178.78 examples/s]
Generating train examples...: 415 examples [00:03, 275.87 examples/s]
Generating train examples...: 540 examples [00:03, 401.78 examples/s]
Generating train examples...: 669 examples [00:03, 538.12 examples/s]
Generating train examples...: 790 examples [00:03, 659.84 examples/s]
Ge

In [23]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

#count occurances of each word
from collections import Counter
vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

#Keep only X most common words
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

#Replace words with encodings adding buckets for words that got dropped out
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

#Create training set
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

#Model and train
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
        input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam",
    metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 64s 76ms/step - loss: 0.6845 - accuracy: 0.5309
Epoch 2/5
782/782 [==============================] - 63s 81ms/step - loss: 0.4539 - accuracy: 0.7846
Epoch 3/5
782/782 [==============================] - 59s 75ms/step - loss: 0.3095 - accuracy: 0.8698
Epoch 4/5
782/782 [==============================] - 55s 71ms/step - loss: 0.1914 - accuracy: 0.9286
Epoch 5/5
782/782 [==============================] - 56s 71ms/step - loss: 0.1458 - accuracy: 0.9473


In [24]:
#Masking Up
K = keras.backend
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)
model = keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(loss="binary_crossentropy", optimizer="adam",
    metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 72s 79ms/step - loss: 0.5990 - accuracy: 0.6515
Epoch 2/5
782/782 [==============================] - 67s 86ms/step - loss: 0.3647 - accuracy: 0.8418
Epoch 3/5
782/782 [==============================] - 69s 88ms/step - loss: 0.2110 - accuracy: 0.9246
Epoch 4/5
782/782 [==============================] - 70s 89ms/step - loss: 0.1341 - accuracy: 0.9529
Epoch 5/5
782/782 [==============================] - 64s 82ms/step - loss: 0.1080 - accuracy: 0.9604


In [26]:
#Pretrained embeddings
import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
        dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
    metrics=["accuracy"])

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].batch(batch_size).prefetch(1)
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 3s 3ms/step - loss: 0.5876 - accuracy: 0.6955
Epoch 2/5
782/782 [==============================] - 2s 3ms/step - loss: 0.5190 - accuracy: 0.7438
Epoch 3/5
782/782 [==============================] - 2s 3ms/step - loss: 0.5135 - accuracy: 0.7467
Epoch 4/5
782/782 [==============================] - 3s 3ms/step - loss: 0.5099 - accuracy: 0.7494
Epoch 5/5
782/782 [==============================] - 2s 3ms/step - loss: 0.5070 - accuracy: 0.7514


In [28]:
#Translation
import tensorflow_addons as tfa

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler,
    output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, initial_state=encoder_state,
    sequence_length=sequence_lengths)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.Model(inputs=[encoder_inputs, decoder_inputs, sequence_lengths], outputs=[Y_proba])

In [30]:
#Beam Searching to correct past mistakes
beam_width = 10
decoder = tfa.seq2seq.beam_search_decoder.BeamSearchDecoder(
    cell=decoder_cell, beam_width=beam_width, output_layer=output_layer)
decoder_initial_state = tfa.seq2seq.beam_search_decoder.tile_batch(
    encoder_state, multiplier=beam_width)
# outputs, _, _ = decoder(
#     embedding_decoder, start_tokens=start_tokens, end_token=end_token,
#     initial_state=decoder_initial_state)


In [32]:
#ATTENTION ATTENTION - Shinedown
# attention_mechanism = tfa.seq2seq.attention_wrapper.LuongAttention(
#     units, encoder_state, memory_sequence_length=encoder_sequence_length)
# attention_decoder_cell = tfa.seq2seq.attention_wrapper.AttentionWrapper(
#     decoder_cell, attention_mechanism, attention_layer_size=n_units)

#Only attention (Transformers)

In [34]:
#Positional Encoding
class PositionalEncoding(keras.layers.Layer):
    def __init__(self, max_steps, max_dims, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        if max_dims % 2 == 1: max_dims += 1 # max_dims must be even
        p, i = np.meshgrid(np.arange(max_steps), np.arange(max_dims // 2))
        pos_emb = np.empty((1, max_steps, max_dims))
        pos_emb[0, :, ::2] = np.sin(p / 10000**(2 * i / max_dims)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10000**(2 * i / max_dims)).T
        self.positional_embedding = tf.constant(pos_emb.astype(self.dtype))
    def call(self, inputs):
        shape = tf.shape(inputs)
        return inputs + self.positional_embedding[:, :shape[-2], :shape[-1]]

embed_size = 512; max_steps = 500; vocab_size = 10000
encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)
positional_encoding = PositionalEncoding(max_steps, max_dims=embed_size)
encoder_in = positional_encoding(encoder_embeddings)
decoder_in = positional_encoding(decoder_embeddings)

Z = encoder_in
for N in range(6):
    Z = keras.layers.Attention(use_scale=True)([Z, Z])

encoder_outputs = Z
Z = decoder_in
for N in range(6):
    Z = keras.layers.Attention(use_scale=True, causal=True)([Z, Z])
    Z = keras.layers.Attention(use_scale=True)([Z, encoder_outputs])

outputs = keras.layers.TimeDistributed(
    keras.layers.Dense(vocab_size, activation="softmax"))(Z)